Bootstrap Routes


In [ ]:
# | default_exp routes.bootstrap

In [ ]:
# | export
import aiohttp

import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda

In [ ]:
# | export
class GetBootstrap_InvalidAuthMethod(Exception):
    def __init__(self, domo_instance):
        message = f"invalid auth method sent to {domo_instance} bootstrap API, use DomoFullAuth (username and password) authentication"
        super().__init__(message)

async def get_bootstrap(
    auth: dmda.DomoFullAuth, ## only works with DomoFullAuth authentication, do not use TokenAuth
    debug_api: bool = False, session: aiohttp.ClientSession = None
) -> rgd.ResponseGetData:
    """get bootstrap data"""

    # url = f"https://{auth.domo_instance}.domo.com/api/domoweb/bootstrap?v2Navigation=false"
    url = f"https://{auth.domo_instance}.domo.com/api/domoweb/bootstrap?v2Navigation=true"

    res = await gd.get_data(
        url=url, method="GET", auth=auth, debug_api=debug_api, session=session
    )

    if res.status == 302:
        raise GetBootstrap_InvalidAuthMethod(auth.domo_instance)

    return res


#### sample implementation of get_boostrap

In [ ]:
import os


auth = dmda.DomoTokenAuth(
    domo_instance='domo-dojo', domo_access_token=os.environ['DOMO_DOJO_ACCESS_TOKEN'])

# auth = dmda.DomoFullAuth(
#     domo_instance='domo-dojo', domo_password = os.environ['DOJO_PASSWORD'], domo_username = 'jae@onyxreporting.com')

try:
    await get_bootstrap(auth = auth, debug_api = False)
    
except GetBootstrap_InvalidAuthMethod as e:
    print(e)

invalid auth method sent to domo-dojo bootstrap API, use DomoFullAuth (username and password) authentication


In [ ]:
import os

full_auth = dmda.DomoFullAuth(
    domo_instance='domo-dojo', domo_password = os.environ['DOJO_PASSWORD'], domo_username = 'jae@onyxreporting.com')

res = await get_bootstrap(auth = full_auth, debug_api = False)
res.response.keys()

dict_keys(['notifierConfig', 'currentUser', 'data'])

In [ ]:
# | export
async def get_bootstrap_features(   
    auth: dmda.DomoAuth, session: aiohttp.ClientSession = None, debug_api: bool = False
) -> rgd.ResponseGetData:
    res = await get_bootstrap(auth=auth, session=session, debug_api=debug_api)

    if not res.is_success:
        return None

    res.response = res.response.get("data").get("features")
    return res

#### sample implementation of get_bootstrap_features

In [ ]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance='domo-dojo', domo_password=os.environ['DOJO_PASSWORD'], domo_username='jae@onyxreporting.com')

res = await get_bootstrap_features(auth=full_auth, debug_api=False)

pd.DataFrame(res.response[0:5])


,id,name,label,description,enabled,purchased,type,environment,owner
0,4,search,,,True,False,STANDARD,,
1,5,launcher,,,True,True,PREMIUM,,
2,9,profile-reminder,,,True,False,STANDARD,,
3,17,enableSwapDatasource,,,True,False,STANDARD,,
4,19,up,,,True,True,STANDARD,,


In [ ]:
# | export
async def get_bootstrap_pages(
    auth: dmda.DomoAuth, session: aiohttp.ClientSession = None, debug_api: bool = False
) -> rgd.ResponseGetData:
    res = await get_bootstrap(auth=auth, session=session, debug_api=debug_api)

    if not res.is_success:
        return None

    res.response = res.response.get("data").get("pages")
    return res


#### sample get bootstrap pages

In [ ]:
import os
import pandas as pd

full_auth = dmda.DomoFullAuth(
    domo_instance='domo-dojo', domo_password=os.environ['DOJO_PASSWORD'], domo_username='jae@onyxreporting.com')

res = await get_bootstrap_pages(auth=full_auth, debug_api=False)

pd.DataFrame(res.response[0:5])


,id,title,locked,sharedView,pageVisible,owner,virtualPage,isValidBadgePage,canAddCard,bibHiddenPage,hasAccess,hierarchy,separator,type,pageLocation,owners,isOwner,childCount,children
0,-100000,Overview,False,False,True,0,True,True,True,False,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-100003,Favorites,False,False,True,0,True,NaN,False,False,True,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-100002,Shared,False,False,True,0,True,NaN,False,False,True,0,True,NaN,NaN,NaN,NaN,NaN,NaN
3,30507758,Welcome,True,True,True,587894148,NaN,NaN,True,False,True,0,NaN,page,0.0,"[{'id': 587894148, 'type': 'USER', 'displayNam...",False,0.0,NaN
4,1267639476,Dojo Solutions,True,True,True,587894148,NaN,NaN,True,False,True,0,NaN,page,1.0,"[{'id': 587894148, 'type': 'USER', 'displayNam...",False,3.0,"[{'id': '1759878295', 'title': 'Beast Modes', ..."


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
